In [ ]:
%%capture
! pip install tokenizers transformers ipywidgets pandas datasets wandb huggingface_hub tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
! pip install accelerate -U

     |████████████████████████████████| 297 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 61.0 MB 18.9 MB/s eta 0:00:01
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
     |████████████████████████████████| 133 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 23.8 MB/s eta 0:00:01
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
You should consider upgrading via the '/Users/tashitsering/Desktop/Work/model/wylie-tokenizer/.env/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

import torch
from torch.utils.data.dataset import Dataset

import os
import math

from huggingface_hub import HfFolder, notebook_login
from datasets import load_dataset

In [5]:
VOCAB_SIZE = 256000
MAX_LEN    = 512

In [7]:
dataset = load_dataset('ta4tsering/Tibetan_transliteration_wylie_datasets')

Repo card metadata block was not found. Setting CardData to empty.


In [9]:
text_data = []
for entry in dataset['train']:  # Adjust the slice as needed to limit data volume
    text_data.append(entry['[text]'])

In [10]:
text_data[0]

"tshar chen blo gsal rgya mtsho'i gsung thor bu/__"

In [12]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel as ByteLevelPreTokenizer

tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.pre_tokenizer = ByteLevelPreTokenizer()

# Define the trainer with the desired specifications
trainer = BpeTrainer(
    vocab_size=VOCAB_SIZE,  # Customize the vocabulary size as needed
    min_frequency=2,  # Minimum frequency for tokens
    special_tokens=["<s>", "</s>", "<pad>", "<unk>", "<mask>"]
)

# Now train the tokenizer
tokenizer.train_from_iterator(text_data, trainer)

In [22]:
import os

# Disable parallelism to avoid potential deadlocks
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

tokenizer_folder = 'tokenizer'
if not os.path.exists(tokenizer_folder):
    os.makedirs(tokenizer_folder)

In [23]:
tokenizer.save(f"{tokenizer_folder}/tokenizer.json")
tokenizer.model.save(tokenizer_folder)

['tokenizer/vocab.json', 'tokenizer/merges.txt']